In [ ]:
# docker file used for this notebook for pytorch 2.0
# docker pull cnstark/pytorch:2.0.1-py3.10.11-cuda11.8.0-ubuntu22.04

In [ ]:
import torch

print(torch.__version__)

2.8.0+cu128


In [ ]:
import torch

print(torch.__version__)

2.0.1+cu118



# Семинар torch.compile 

Cеминар основан на документации [1] и ноутбуке [2].<br>

[1] https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html <br>
[2] https://colab.research.google.com/github/PyTorchKorea/tutorials-kr/blob/master/docs/_downloads/96ad88eb476f41a5403dcdade086afb8/torch_compile_tutorial.ipynb <br>

``torch.compile`` компилятор для ускорения кода на PyTorch. <br>
Это JIT-компилятор с использованием оптимизированных кернелей,
который требует минимального изменения кода. <br>

В этом семинаре мы рассмотрим базовое использование ``torch.compile``, а так же сравним его с [TorchScript](https://pytorch.org/docs/stable/jit.html) и
[FX Tracing](https://pytorch.org/docs/stable/fx.html#torch.fx.symbolic_trace).

**Содержание**

- Базовый пример
- Ускорение с помощью ``torch.compile``
- Сравнение с TorchScript и FX Tracing
- Заключение

**Required pip Dependencies**

- ``torch >= 2.0``
- ``torchvision``
- ``numpy``
- ``scipy``
- ``tabulate``

docker file used for this notebook for pytorch 2.0: <br>
``docker pull cnstark/pytorch:2.0.1-py3.10.11-cuda11.8.0-ubuntu22.04``





ПРИМЕЧАНИЕ. Результаты зависят от версии GPU, что бы воспроизвести результаты команда ``torch.compile`` рекомендует использование NVIDIA (H100, A100 или V100). 

## Базовый пример

Произвольные функции Python можно оптимизировать, передав вызываемый объект в
``torch.compile``. Затем мы можем вызвать возвращенную оптимизированную
функцию вместо исходной.



In [ ]:
import torch


def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b


opt_foo = torch.compile(foo, mode="reduce-overhead")

print(opt_foo(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[-0.1337,  0.2004,  0.7494,  0.1009,  1.8179,  1.6750,  0.4611,  1.5575,
         -0.1934,  1.1764],
        [-0.0113,  0.1501,  0.3848,  0.9656,  1.8039,  1.4148,  0.8667,  0.1797,
          1.5050,  1.9274],
        [-0.4040,  0.2448,  0.4371,  1.8861,  0.1530,  1.5537,  0.2199,  0.0800,
          0.5754,  1.3568],
        [-0.2244,  0.4271,  1.5272,  1.0350, -0.2149, -0.0359, -1.2749,  1.5528,
          1.2313, -0.1416],
        [ 1.2344, -0.4453,  0.1077,  0.7902, -0.6938,  0.1555,  0.4434,  0.4753,
          1.0193, -0.9368],
        [ 1.5855,  0.9523,  1.1757,  1.7563,  0.0637,  0.8269,  0.2546,  0.3878,
         -0.0047,  0.3956],
        [-1.2149,  1.3964,  1.0780,  0.7782,  0.9994,  0.2362,  1.2506,  0.9998,
         -0.8594,  1.2726],
        [ 1.8122, -1.6441,  1.3831,  0.6697,  0.3341,  1.3493, -0.2486,  1.0417,
         -1.1308,  0.2741],
        [ 1.9186, -0.4975, -0.1375,  0.1934,  0.2986,  0.7905,  0.1585, -0.0749,
          1.1216,  0.0102],
        [-1.3142,  

/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:279: UserWarning: changing options to `torch.compile()` may require calling `torch._dynamo.reset()` to take effect
  warnings.warn(


Так же можно использовать декоратор

In [ ]:
@torch.compile
def opt_foo2(x, y):
    a = torch.sin(x)
    b = torch.cos(x)
    return a + b


print(opt_foo2(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[-0.2478,  1.3776,  0.4187,  0.3296,  1.3416,  1.0529,  0.7627,  0.9105,
         -0.0590, -0.8596],
        [-0.4838,  1.3639, -0.7595,  0.1312, -0.4861,  1.1997,  1.4128, -1.3991,
          1.2488,  0.4051],
        [-0.2830,  0.9390,  1.0648, -0.8903,  0.5795,  1.0742,  0.7712,  1.3832,
          1.1962, -0.4351],
        [ 0.6838, -0.1808,  1.2921, -0.2147, -0.1757,  0.4896,  1.4141, -0.4459,
          1.4132,  1.3821],
        [ 1.3422,  0.7348,  1.0283, -1.0379,  1.3280, -0.5364, -0.5430,  0.5936,
          0.1113,  1.3315],
        [-0.0918,  0.0093,  0.3959,  1.2918,  0.9686, -0.5804,  1.4059, -0.1153,
          0.7315,  1.4139],
        [ 1.4055,  1.2292,  0.9978, -1.0161,  1.0527, -1.4117,  0.7336,  0.5024,
         -0.7475,  0.0015],
        [-0.3200,  1.0785,  1.2201,  0.9576,  1.3288,  0.8020,  0.8402, -0.0572,
         -0.3814,  1.3116],
        [ 1.1942,  0.7931,  1.1332,  1.2324, -1.1348,  0.7660,  1.3582,  0.7928,
         -0.6528,  1.0945],
        [ 0.2760, -

/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:279: UserWarning: changing options to `torch.compile()` may require calling `torch._dynamo.reset()` to take effect
  warnings.warn(


Мы можем оптимизировать целый модуль ``torch.nn.Module``.



In [ ]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = torch.nn.Linear(100, 10)

    def forward(self, x):
        return torch.nn.functional.relu(self.lin(x))


mod = MyModule()

opt_mod = torch.compile(mod)
print(opt_mod(torch.randn(10, 100)))

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6326, 0.8305, 2.2030,
         0.2086],
        [0.8224, 0.0000, 0.8526, 0.0000, 0.4102, 1.3600, 0.0000, 0.0000, 0.2953,
         0.0000],
        [0.0000, 0.0000, 1.2665, 0.4916, 0.0000, 1.7656, 0.4085, 0.0000, 0.7538,
         0.0000],
        [0.5447, 0.0000, 0.5230, 0.0000, 0.0000, 1.2311, 0.0000, 0.0000, 0.1864,
         0.0503],
        [0.2599, 0.0000, 0.0000, 0.0000, 1.2592, 0.0588, 0.0000, 0.1733, 0.0000,
         0.0000],
        [0.0000, 0.1221, 0.0000, 0.1661, 0.0000, 0.4178, 0.4947, 0.5990, 0.1609,
         1.1907],
        [0.0000, 0.3557, 0.0000, 0.0000, 0.0000, 0.0000, 0.3288, 0.1071, 0.6306,
         0.0000],
        [0.0000, 0.0000, 0.6321, 0.7258, 0.0000, 0.0000, 0.0000, 0.0000, 1.0887,
         0.1276],
        [0.2477, 0.0000, 0.0000, 0.0000, 0.2789, 0.4192, 0.0993, 0.0000, 0.1152,
         0.3099],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1485, 0.4526, -0.0000, 0.2448, -0.0000,
         -0.0000]], grad_f


## Ускорение для инференса

Давайте теперь посмтрим, как использование ``torch.compile`` может ускорить <br>
модели. Мы сравним стандартный eager режим и ``torch.compile`` для инфиренса и для обучения ResNet-18 на случайных данных.

Но прежде чем мы начнем, нам нужно определить некоторые служебные функции.


In [ ]:
# Returns the result of running `fn()` and the time it took for `fn()` to run,
# in seconds. We use CUDA events and synchronization for the most accurate
# measurements.
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000


# Generates random input and targets data for the model, where `b` is
# batch size.
def generate_data(b):
    return (
        torch.randn(b, 3, 128, 128).to(torch.float32).cuda(),
        torch.randint(1000, (b,)).cuda(),
    )


from torchvision.models import densenet121


def init_model():
    return densenet121().to(torch.float32).cuda()

Первым делом мы сравним инфиренс, но прежде посмотрим сколько времени занимает компиляция.

В  ``torch.compile`` мы используем некотрые параметры для переменной ``mode``, мы обсудим их позже.


In [ ]:
def evaluate(mod, inp):
    return mod(inp)


model = init_model()

# Reset since we are using a different model.
import torch._dynamo

torch._dynamo.reset()


evaluate_opt1 = torch.compile(evaluate, mode="reduce-overhead")
evaluate_opt2 = torch.compile(evaluate, mode="max-autotune")


inp = generate_data(16)[0]

# One Iteration for each function
print("reduce-overhead:", timed(lambda: evaluate_opt1(model, inp))[1])
print("max-autotune:", timed(lambda: evaluate_opt2(model, inp))[1])
print("original", timed(lambda: evaluate(model, inp))[1])

reduce-overhead: 64.14381640625
max-autotune: 0.012602720260620117
original 0.03690089416503906


/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:279: UserWarning: changing options to `torch.compile()` may require calling `torch._dynamo.reset()` to take effect
  warnings.warn(


In [ ]:
# One Iteration for each function
print("reduce-overhead:", timed(lambda: evaluate_opt1(model, inp))[1])
print("max-autotune:", timed(lambda: evaluate_opt2(model, inp))[1])
print("original", timed(lambda: evaluate(model, inp))[1])

reduce-overhead: 0.015678784370422364
max-autotune: 0.012794303894042968
original 0.03194457626342773


Обратите внимание, что выполнение ``torch.compile`` занимает намного больше времени <br>
по сравнению с ``eager mode``. Это потому, что ``torch.compile`` занимается компиляцией и  <br>
оптимизирует ядра по мере ее выполнения.  Здесь мы запускаем нашу модель один раз. <br>

Если мы будем запускать нашу модель несколько раз, то после компиляции, которая происходит на первом этапе мы увидим ускорение. <br>

In [ ]:
N_ITERS = 10

eager_times = []
compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    _, eager_time = timed(lambda: evaluate(model, inp))
    eager_times.append(eager_time)
    print(f"eager eval time {i}: {eager_time}")

print("~" * 10)

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    _, compile_time = timed(lambda: evaluate_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile eval time {i}: {compile_time}")
print("~" * 10)

import numpy as np

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
print(f"(eval) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager eval time 0: 0.02985795211791992
eager eval time 1: 0.027883712768554687
eager eval time 2: 0.029763904571533203
eager eval time 3: 0.028658912658691405
eager eval time 4: 0.031525152206420895
eager eval time 5: 0.029459903717041016
eager eval time 6: 0.031011680603027343
eager eval time 7: 0.027303936004638672
eager eval time 8: 0.02947148895263672
eager eval time 9: 0.027056192398071287
~~~~~~~~~~
compile eval time 0: 0.012481087684631348
compile eval time 1: 0.012455840110778809
compile eval time 2: 0.012064767837524413
compile eval time 3: 0.011994784355163574
compile eval time 4: 0.012480095863342285
compile eval time 5: 0.012183551788330077
compile eval time 6: 0.012075167655944824
compile eval time 7: 0.012062432289123536
compile eval time 8: 0.012034111976623536
compile eval time 9: 0.012016351699829102
~~~~~~~~~~
(eval) eager median: 0.029465696334838868, compile median: 0.01206996774673462, speedup: 2.441240685403691x
~~~~~~~~~~


In [ ]:
N_ITERS = 10

model = init_model()
opt = torch.optim.Adam(model.parameters())


def train(mod, data):
    opt.zero_grad(True)
    pred = mod(data[0])
    loss = torch.nn.CrossEntropyLoss()(pred, data[1])
    loss.backward()
    opt.step()


eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, eager_time = timed(lambda: train(model, inp))
    eager_times.append(eager_time)
    print(f"eager train time {i}: {eager_time}")
print("~" * 10)

model = init_model()
opt = torch.optim.Adam(model.parameters())
train_opt = torch.compile(train, mode="reduce-overhead")

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, compile_time = timed(lambda: train_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile train time {i}: {compile_time}")
print("~" * 10)

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
print(f"(train) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager train time 0: 0.24858819580078126
eager train time 1: 0.07555916595458985
eager train time 2: 0.07133321380615235
eager train time 3: 0.0655093765258789
eager train time 4: 0.06588352203369141
eager train time 5: 0.06761344146728515
eager train time 6: 0.06586534118652344
eager train time 7: 0.06962089538574219
eager train time 8: 0.06684508514404297
eager train time 9: 0.06550166320800781
~~~~~~~~~~
compile train time 0: 90.48184375
compile train time 1: 0.045564224243164066
compile train time 2: 0.035632991790771486
compile train time 3: 0.03702067184448242
compile train time 4: 0.03566387176513672
compile train time 5: 0.03539820861816406
compile train time 6: 0.03708006286621094
compile train time 7: 0.036988479614257816
compile train time 8: 0.03706675338745117
compile train time 9: 0.03610502243041992
~~~~~~~~~~
(train) eager median: 0.06722926330566406, compile median: 0.03700457572937012, speedup: 1.8167824378622708x
~~~~~~~~~~


In [ ]:
compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, compile_time = timed(lambda: train_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile train time {i}: {compile_time}")
print("~" * 10)

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
print(f"(train) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

compile train time 0: 0.043170143127441406
compile train time 1: 0.04607766342163086
compile train time 2: 0.03811705780029297
compile train time 3: 0.041971038818359375
compile train time 4: 0.0421673583984375
compile train time 5: 0.04322812652587891
compile train time 6: 0.04122175979614258
compile train time 7: 0.04205740737915039
compile train time 8: 0.04366767883300781
compile train time 9: 0.04351142501831055
~~~~~~~~~~
(train) eager median: 0.06722926330566406, compile median: 0.04266875076293945, speedup: 1.5756088965242683x
~~~~~~~~~~


Опять, мы видим, что ``torch.compile`` первая итерация занимает больше времени.
итерации, так как она должна скомпилировать модель, но на последующих итерациях мы видим
значительное ускорение по сравнению с нетерпеливым.


## Cравнение с TorchScript и FX Tracing

Мы видели что ``torch.compile`` может ускорить вычисления. <br>


Прежде всего, преимущество ``torch.compile`` заключается в его способности работать
почти с произвольным кодом на Python с его минимальными изменениями. <br>

А так же ``torch.compile`` может работать с data-dependent control flow ``if x.sum() < 0:``.



In [ ]:
def f1(x):
    if x < 10:
        return torch.tensor(0.0)
    return x


# Test that `fn1` and `fn2` return the same result, given
# the same arguments `args`. Typically, `fn1` will be an eager function
# while `fn2` will be a compiled function (torch.compile, TorchScript, or FX graph).
def test_fns(fn1, fn2, args):
    out1 = fn1(*args)
    out2 = fn2(*args)
    return print(out1, out2)


inp1 = torch.tensor(5)
inp2 = torch.tensor(15)

TorchScript tracing ``f1`` привдет к неправильным результатам, так  ак зафиксируется детерменированный проход по данным.



In [ ]:
traced_f1 = torch.jit.trace(f1, (inp1))
f1(inp1), traced_f1(inp1)

/tmp/ipykernel_99144/2755518421.py:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x < 10:
/tmp/ipykernel_99144/2755518421.py:3: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.tensor(0.)


(tensor(0.), tensor(0.))

In [ ]:
f1(inp2), traced_f1(inp2)

(tensor(15), tensor(0.))

```
TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() < 0:
```

FX tracing ``f1`` приводит к ошибке из-за присутствия поток управления, зависящий от данных.



In [ ]:
import traceback as tb

try:
    torch.fx.symbolic_trace(f1)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_99144/1451191637.py", line 3, in <module>
    torch.fx.symbolic_trace(f1)
  File "/usr/local/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py", line 1109, in symbolic_trace
    graph = tracer.trace(root, concrete_args)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 209, in _fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py", line 778, in trace
    (self.create_arg(fn(*args)),),
  File "/tmp/ipykernel_99144/2755518421.py", line 2, in f1
    if x < 10:
  File "/usr/local/lib/python3.10/site-packages/torch/fx/proxy.py", line 413, in __bool__
    return self.tracer.to_bool(self)
  File "/usr/local/lib/python3.10/site-packages/torch/fx/proxy.py", line 276, in to_bool
    raise TraceError('symbolically traced variables cannot be used as inputs to control flow')
torch.fx.proxy.TraceError: symbolically traced variables cannot be used 

Если мы предоставим значение для ``x`` при попытке  FX  trace ``f1``, тогда
мы сталкиваемся с той же проблемой, что и при TorchScript, из за зависимости графа от данных.

In [ ]:
fx_f1 = torch.fx.symbolic_trace(f1, concrete_args={"x": inp1})
fx_f1(inp1), fx_f1(inp2)

/usr/local/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py:602: UserWarning: Was not able to add assertion to guarantee correct input x to specialized function. It is up to the user to make sure that your inputs match the inputs you specialized the function with.
  warnings.warn(


(tensor(0.), tensor(0.))

В это время ``torch.compile`` корректно обрабатывает динамический control-flow.



In [ ]:
# Reset since we are using a different mode.
torch._dynamo.reset()

compile_f1 = torch.compile(f1)
compile_f1(inp1), compile_f1(inp2)

(tensor(0.), tensor(15))

TorchScript scripting может работать в этой ситуации, но нам нужно будет адаптировать код и убедиться что у нас используется статическое типизирование данных.



In [ ]:
def f2(x, y):
    return x + y


inp1 = torch.randn(5, 5)
inp2 = 3

script_f2 = torch.jit.script(f2)
try:
    script_f2(inp1, inp2)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_99144/895162831.py", line 9, in <module>
    script_f2(inp1, inp2)
RuntimeError: f2() Expected a value of type 'Tensor (inferred)' for argument 'y' but instead found type 'int'.
Inferred 'y' to be of type 'Tensor' because it was not annotated with an explicit type.
Position: 1
Value: 3
Declaration: f2(Tensor x, Tensor y) -> Tensor
Cast error details: Unable to cast 3 to Tensor


In [ ]:
def f2(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return x + y


inp2 = torch.tensor(3.0)


script_f2 = torch.jit.script(f2)
try:
    script_f2(inp1, inp2)
except:
    tb.print_exc()

Другое преимущество ``torch.compile``  это возможность использлвать больше функций.


In [ ]:
import scipy
import torch


def f3(x):
    x *= 2
    x = x.numpy()
    x = torch.from_numpy(x)
    x *= 2
    return x


traced_f3 = torch.jit.trace(f3, torch.tensor([1, 2, 3]))

/var/tmp/ipykernel_2818737/1543673579.py:7: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = x.numpy()
/var/tmp/ipykernel_2818737/1543673579.py:8: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.from_numpy(x)
/home/deniskirbaba/miniconda3/envs/ml/lib/python3.13/site-packages/torch/jit/_trace.py:789: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3 / 3 (100.0%)
Greatest abso

TracingCheckError: Tracing failed sanity checks!
ERROR: Graphs differed across invocations!
	Graph diff:
		  graph(%x.1 : Tensor):
		    %1 : Tensor = prim::Constant[value={2}]() # /var/tmp/ipykernel_2818737/1543673579.py:6:0
		    %x.3 : Tensor = aten::mul_(%x.1, %1) # /var/tmp/ipykernel_2818737/1543673579.py:6:0
		-   %3 : Tensor = prim::Constant[value=  8  16  24 [ CPULongType{3} ]]() # /var/tmp/ipykernel_2818737/1543673579.py:8:0
		?                                       ----
		+   %3 : Tensor = prim::Constant[value= 16  32  48 [ CPULongType{3} ]]() # /var/tmp/ipykernel_2818737/1543673579.py:8:0
		?                                           + ++ +
		    %x : Tensor = aten::lift_fresh(%3) # /var/tmp/ipykernel_2818737/1543673579.py:8:0
		    %5 : Tensor = prim::Constant[value={2}]() # /var/tmp/ipykernel_2818737/1543673579.py:9:0
		    %6 : Tensor = aten::mul_(%x, %5) # /var/tmp/ipykernel_2818737/1543673579.py:9:0
		    return (%6)
	First diverging operator:
	Node diff:
		- %3 : Tenso

In [ ]:
f3(torch.tensor([9])), traced_f3(torch.tensor([9]))

(tensor([36]), tensor([36]))

In [ ]:
import scipy


def f3(x):
    x = x * 2
    x = scipy.fft.dct(x.numpy())
    x = torch.from_numpy(x)
    x = x * 2
    return x

TorchScript обрабатывает результаты вызовов функций, отличных от PyTorch.
как константы, и поэтому результаты могут быть ошибочными.

In [ ]:
inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)
traced_f3 = torch.jit.trace(f3, (inp1,))
print("traced 3:", np.allclose(f3(inp2), traced_f3(inp2)))

traced 3: False


/tmp/ipykernel_99144/2558593883.py:5: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = scipy.fft.dct(x.numpy())
/tmp/ipykernel_99144/2558593883.py:6: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.from_numpy(x)


In [ ]:
inp1 = torch.randn(3, 3)
traced_f3 = torch.jit.trace(f3, (inp1,))
traced_f3(torch.tensor(2.0))

/tmp/ipykernel_99144/2558593883.py:5: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = scipy.fft.dct(x.numpy())
/tmp/ipykernel_99144/2558593883.py:6: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.from_numpy(x)


tensor([[ 2.0957, -6.4813, -7.0046],
        [-2.2467, 10.3532, -2.3497],
        [ 3.6110, -6.7818,  3.8129]])

``torch.compile`` 


In [ ]:
inp2 = torch.randn(2, 2)
compile_f3 = torch.compile(f3)
compile_f3(inp1), compile_f3(inp2)

(tensor([[-32.0297,  -9.5198,  -3.5707,  -7.3001, -13.0426],
         [ 18.7543,  22.0072,  -5.5431, -13.7117, -10.2907],
         [-17.4517,   3.1124,  17.8939,  17.0423,  -8.9267],
         [ 25.0059,   6.2932,  -6.2911,  18.3122,  -4.5237],
         [ -2.1596,   0.2294,  -9.7409,  -2.7702, -16.6107]]),
 tensor([[ -3.6311, -15.0039],
         [-12.4913,  -1.7700]]))

## TorchDynamo и FX Graphs

Одна из важных компонент ``torch.compile`` это TorchDynamo.
TorchDynamo отвечает за JIT компиляцию кода на Python в
[FX graphs](https://pytorch.org/docs/stable/fx.html#torch.fx.Graph), который далее может быть оптимизирован. TorchDynamo выделяет FX графы анализруя Питоновский байткод во время выполнения, а так же отслеживает вызовы к функциям PyTorch.

TorchInductor, другая компонента ``torch.compile``,
конвертирует FX граф в оптимизированные кернели, а
TorchDynamo позволяет использовать различные бекнеды.  <br>


Давайте создадим свой бекенд, который выводит FX граф и не оптимизированный forward (что-то типа принта, но вызывывается он TorchDynamo). 

In [ ]:
from typing import List


def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("custom backend called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward


# Reset since we are using a different backend.
torch._dynamo.reset()

opt_model = torch.compile(init_model(), backend=custom_backend)
opt_model(generate_data(16)[0])

custom backend called with FX graph:
opcode         name                                 target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

tensor([[-0.0228, -0.1196,  0.1445,  ..., -0.0912, -0.0534, -0.4670],
        [-0.0670, -0.1353,  0.1420,  ...,  0.1107, -0.1541, -0.3582],
        [-0.1230, -0.0106,  0.1470,  ..., -0.0431,  0.0839, -0.3706],
        ...,
        [-0.1057, -0.0944, -0.0449,  ...,  0.0021, -0.0412, -0.3044],
        [ 0.0338,  0.0361,  0.1123,  ...,  0.0336, -0.0731, -0.3573],
        [ 0.0807, -0.0464,  0.1297,  ...,  0.1894, -0.0607, -0.3836]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


Используя наш собственный бэкэнд, мы теперь можем увидеть, как TorchDynamo работает с ситуацией, когда есть зависимость от данных. Рассмотрим:
``if b.sum() < 0``. 



In [ ]:
def bar(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b


opt_bar = torch.compile(bar, backend=custom_backend)
inp1 = torch.randn(10)
inp2 = torch.randn(10)
opt_bar(inp1, inp2)
opt_bar(inp1, -inp2)

custom backend called with FX graph:
opcode         name     target                                                  args              kwargs
-------------  -------  ------------------------------------------------------  ----------------  --------
placeholder    a        a                                                       ()                {}
placeholder    b        b                                                       ()                {}
call_function  abs_1    <built-in method abs of type object at 0x7fe4c9aa4880>  (a,)              {}
call_function  add      <built-in function add>                                 (abs_1, 1)        {}
call_function  truediv  <built-in function truediv>                             (a, add)          {}
call_method    sum_1    sum                                                     (b,)              {}
call_function  lt       <built-in function lt>                                  (sum_1, 0)        {}
output         output   output              

tensor([ 0.0950,  0.3052,  0.0031,  0.8163, -0.4258,  0.5262, -0.0300,  0.4338,
        -0.0761,  0.3774])

Мы видим что TorchDynamo разбил наш код на три части, которые соотвествуют:


1. ``x = a / (torch.abs(a) + 1)``
2. ``b = b * -1; return x * b``
3. ``return x * b``

Когда TorchDynamo встречает не поддерживаемые в функции, например зависящие от входных данных, он  разбивает код на части, дает передает все в стандатный Python интерпритатор, а потом возвращается к графу.

Давайте на примере разберемся, как TorchDynamo пройдет через ``<``.
Если ``b.sum() < 0``, то TorchDynamo запустит граф 1, даст
Python определить результат условного выражения, а затем запустит
граф 2. С другой стороны, если ``не b.sum() < 0``, то TorchDynamo
запустил бы граф 1, позволил Python определить результат условного выражения, затем
запустил график 3. <br>


Это подчеркивает основное различие между TorchDynamo и предыдущим  компиляторами для PyTorch.

Предыдущие решения либо упадут с ошибков, либо не правильно скомпилируются и ничего не произойдет.
TorchDynamo, с другой стороны, просто разобьет граф вычислений на части.

Что бы посмотреть как TorchDynamo разбивает граф на части мы можем вызвать: ``torch._dynamo.explain``



In [ ]:
# Reset since we are using a different backend.
torch._dynamo.reset()
a = torch._dynamo.explain(bar, torch.randn(10), torch.randn(10))

Чтобы максимизировать ускорение, разрывы графика должны быть ограничены.
Мы можем заставить TorchDynamo выдавать ошибку на первом разрыве графа ``fullgraph=True``:


In [ ]:
print(a[-1])

Dynamo produced 2 graphs with 1 graph break and 6 ops
 Break reasons: 

1. generic_jump TensorVariable()
  File "/tmp/ipykernel_99144/3263660924.py", line 3, in bar
    if b.sum() < 0:
 
2. return_value
  File "/tmp/ipykernel_99144/3263660924.py", line 5, in <graph break in bar>
    return x * b
 
TorchDynamo compilation metrics:
Function                        Runtimes (s)
------------------------------  --------------
_compile                        0.0218, 0.0088
OutputGraph.call_user_compiler  0.0001, 0.0001


И действительно, мы видим, что запуск нашей модели с помощью ``torch.compile``
приводит к значительному ускорению. Ускорение в основном достигается за счет сокращения накладных расходов Python и
Чтение/запись графического процессора, поэтому наблюдаемое ускорение может варьироваться в зависимости от таких факторов, как 
архитектура модели и ее размер. <br>

<br>
Например, если архитектура модели простя, 
но занимает много памяти, то узким местом будет 
вычисления графического процессора и наблюдаемое ускорение может быть менее значительными. <br>


``torch.compile`` поддерживает разные режими компиляции. Подробнее про разные режимы можно посмотреть тут: [here](https://pytorch.org/get-started/pytorch-2.0/#user-experience).


```python
# default: optimizes for large models, low compile-time
#          and no extra memory usage
torch.compile(model)

# reduce-overhead: optimizes to reduce the framework overhead
#                and uses some extra memory. Helps speed up small models
torch.compile(model, mode="reduce-overhead")

# max-autotune: optimizes to produce the fastest model,
#               but takes a very long time to compile
torch.compile(model, mode="max-autotune")
```



В целом, для тестирования моделей PyTorch лучше использовать ``torch.utils.benchmark`` вместо самопальной timed. <br>

<br>

В этом примере нам нужна была такая функция что бы показать задержки на компиляцию.

<br>
<br>

**Посмотрим на сравнение скорости обучения.**


In [ ]:
opt_bar = torch.compile(bar, fullgraph=True)
try:
    opt_bar(torch.randn(10), torch.randn(10))
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_99144/3610564610.py", line 3, in <module>
    opt_bar(torch.randn(10), torch.randn(10))
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 209, in _fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 337, in catch_errors
    return callback(frame, cache_size, hooks)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 104, in _fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 262, in _convert_frame_assert
    return _compile(
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/utils.py", line 163, in time_wrapper
    r = func(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 324, in _compile
    out_code = transform_code_object(code, transform)
  File "/usr/local/lib/pyt

TorchDynamo не ломает граф модели, который мы использовали для анализа ускорения.

In [ ]:
opt_model = torch.compile(init_model(), fullgraph=True)
print(opt_model(generate_data(16)[0]))

tensor([[ 0.2171, -0.0747,  0.4752,  ...,  0.0521, -0.3713,  0.0506],
        [ 0.0240, -0.0850,  0.5524,  ...,  0.2055, -0.4584, -0.2629],
        [ 0.0528,  0.0457,  0.4437,  ...,  0.0864, -0.5564, -0.0323],
        ...,
        [ 0.2383, -0.0734,  0.4071,  ...,  0.0932, -0.3258,  0.0080],
        [ 0.1479,  0.0311,  0.5479,  ..., -0.0831, -0.6256, -0.0541],
        [ 0.2836,  0.0376,  0.4323,  ...,  0.2145, -0.3397,  0.0143]],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>)


Наконец, если мы просто хотим, чтобы TorchDynamo выдал на torch FX граф,
мы можем использовать torch._dynamo.export. Обратите внимание, что ``torch._dynamo.export`` с
``fullgraph=True``, выдает ошибку, если TorchDynamo находит место разрывы графа.